In [ ]:
import numpy as np
import resource

# Norm function
def norm(v):
    return np.sqrt(np.dot(v, v))

#conjugate directions method
def conjDir(A, b):
    #initialize a,b,n,x,r0,p,residual
    A = np.array(A, float)
    b = np.array(b, float)
    n = b.size
    x = np.zeros(n)
    r0 = b - A.dot(x)
    p = r0.copy()
    residual = [np.linalg.norm(r0)]
    #do 500 iters
    for i in range(1, n+1):
        #dot products
        Ap = A.dot(p)
        rsquare = r0.dot(r0)
        #div
        a = rsquare / p.dot(Ap)
        x = x + a * p
        #update 
        r1 = r0 - a * Ap
        #add to vec
        residual.append(np.linalg.norm(r1))
        beta = r1.dot(r1) / rsquare
        p = r1 + beta * p
        #reset r
        r0 = r1
    return x, i, np.array(residual)

#gaussian elimination method
def gauss(A, b):
    #initialize a,b,n,aug
    A = np.array(A, float)
    b = np.array(b, float)
    n = b.size
    aug = np.zeros((n, n+1), float)
    #copy A to aug
    aug[:, :n] = A
    for i in range(n):
        aug[i, n] = b[i]
    #loop through rows, pivoting and eliminating
    for i in range(n-1):
        #find pivot
        piv = np.argmax(abs(aug[i:, i])) + i
        if piv != i:
            aug[[i, piv]] = aug[[piv, i]]
        #scale row
        for j in range(i+1, n):
            scale = aug[j, i] / aug[i, i]
            #pivot here
            aug[j, i:] -= scale * aug[i, i:]
    #back substitution
    x = np.zeros(n, float)
    for i in range(n-1, -1, -1):
        x[i] = (aug[i, n] - aug[i, i+1:n].dot(x[i+1:n])) / aug[i, i]
    return x

#built in numpy solver for norm proof of concept
#takes in matrix A and vector b, outputting solution vector x, used for comparison to Gauss
def solveDir(A, b):
    A = np.array(A, float)
    b = np.array(b, float)
    #basic solve here for sanity check
    return np.linalg.solve(A, b)

#measure memory, taking in each function and arguments, returns peak memory use
def memory(func, *args):
    #memory pre
    pre = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss
    #run it
    result = func(*args)
    #memory post run
    post = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss
    return result, post - pre

#main 
if __name__ == "__main__":
    #generate random matrix and vector
    M = np.random.randn(500, 500)
    A = M.T.dot(M) + np.eye(500) * 1e-3
    b = np.random.randn(500)
    
    #run solvers through memory function as shell
    (resConj, memConj) = memory(conjDir, A, b)
    xConj, iter, residual = resConj
    (xGauss, memGauss) = memory(gauss, A, b)
    (xDir, memDir) = memory(solveDir, A, b)

    #print results
    print(f"Conjugate Directions: iterations = {iter}, final residual = {residual[-1]:.2e}, Memory use = {memConj} kb ")
    print(f"Gaussian Elimination: final residual = {norm(A.dot(xGauss)-b):.2e} Memory use = {memGauss} kb")
    print(f"Built-in solve: final residual = {norm(A.dot(xDir)-b):.2e}, Memory use = {memDir} kb")



Conjugate Directions: iterations = 500, final residual = 3.93e+00, Memory use = 0 kb 
Gaussian Elimination: final residual = 4.93e-11 Memory use = 0 kb
Built-in solve: final residual = 5.84e-11, Memory use = 16384 kb
